In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="./llm-dungeon-adventures/.env")

True

In [1]:
from environment.game import Game
from environment.generators.random_grid_generator import RandomGridGenerator
from environment.grid_visualizer import GridVisualizer
from environment.types import Direction

DUNGEON_SIZE, generator_seed = 50, 100
generator = RandomGridGenerator(DUNGEON_SIZE, generator_seed)
game = Game(generator)
text = "go north\ngo east\ngo east"
game.play(text)


ModuleNotFoundError: No module named 'environment'